![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQutKA-oYIBw8OtYhNsrZ9uGr87_j_t4QXx-g&usqp=CAU)youtube.com

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#Code by Shashwat Tiwari  https://www.kaggle.com/shashwatwork/insurance-company-complaint-prediction/notebook

In [ ]:
from sklearn.compose import make_column_transformer,ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold,cross_val_score,RepeatedStratifiedKFold,train_test_split
from sklearn.preprocessing import RobustScaler,StandardScaler,LabelEncoder,LabelBinarizer
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from numpy import absolute,mean,std
from imblearn.over_sampling import SMOTE,ADASYN
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score,auc
from imblearn.ensemble import BalancedRandomForestClassifier,EasyEnsembleClassifier,BalancedBaggingClassifier
from sklearn.metrics import classification_report,f1_score,roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RepeatedStratifiedKFold,GridSearchCV,RandomizedSearchCV,StratifiedKFold,cross_val_score
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn.gaussian_process import GaussianProcessClassifier
from xgboost import XGBClassifier

# for model explanation
import shap 
import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
nRowsRead = 1000 # specify 'None' if want to read whole file
df = pd.read_csv('../input/cusersmarildownloadsgermancsv/german.csv', delimiter=';', encoding = "ISO-8859-2", nrows = nRowsRead)
df.dataframeName = 'german.csv'
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')
df.head()

In [ ]:
df1 = df.copy()
scores = pd.DataFrame(columns=['Model', 'Score'])
scores_ohe = pd.DataFrame(columns=['Model', 'Score'])

In [ ]:
x = df1.drop(['Creditability'], axis = 1)
y = df1['Creditability']

In [ ]:
from sklearn.model_selection import KFold,cross_val_score,RepeatedStratifiedKFold,train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

In [ ]:
train_cols_list = X_train.columns.values.tolist()

In [ ]:
from collections import Counter
print('Classes and number of values in trainset',Counter(y_train))

In [ ]:
#Fixing up Imbalance Classes with AdaSyn

from imblearn.over_sampling import SMOTE,ADASYN

oversample = ADASYN()
X_train,y_train = oversample.fit_resample(X_train,y_train)
print('Classes and number of values in trainset after ADSYN:',Counter(y_train))

In [ ]:
from sklearn.preprocessing import RobustScaler,StandardScaler,LabelEncoder,LabelBinarizer

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_pred, average=average)

In [ ]:
model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)

lrpred = model_lr.predict(X_test)

print("Training Accuracy: ", model_lr.score(X_train, y_train))
print('Testing Accuarcy: ', model_lr.score(X_test, y_test))
print('F1 Score',f1_score(y_test, lrpred,average = 'weighted'))
roc_score = multiclass_roc_auc_score(y_test,lrpred)
print("ROC AUC Score - ",roc_score)


# confusion matrix
cm = confusion_matrix(y_test, lrpred)
plt.rcParams['figure.figsize'] = (5, 5)
sns.heatmap(cm, annot = True, cmap = 'rainbow')
plt.show()

# classification report
cr = classification_report(y_test, lrpred)
print(cr)
scores = scores.append({'Model': 'Logistic Regression', 'Score': roc_score}, ignore_index=True)

In [ ]:
model_lr_cv = LogisticRegressionCV(cv=10)
model_lr_cv.fit(X_train, y_train)

lrpred_cv = model_lr_cv.predict(X_test)

print("Training Accuracy: ", model_lr_cv.score(X_train, y_train))
print('Testing Accuarcy: ', model_lr_cv.score(X_test, y_test))
print('F1 Score',f1_score(y_test, lrpred_cv,average = 'weighted'))
roc_score = multiclass_roc_auc_score(y_test,lrpred_cv)
print("ROC AUC Score - ",roc_score)


# confusion matrix
cm = confusion_matrix(y_test, lrpred_cv)
plt.rcParams['figure.figsize'] = (5, 5)
sns.heatmap(cm, annot = True, cmap = 'viridis')
plt.show()

# classification report
cr = classification_report(y_test, lrpred_cv)
print(cr)
scores = scores.append({'Model': 'Logistic RegressionCV', 'Score': roc_score}, ignore_index=True)

In [ ]:
model_nb = GaussianNB()
model_nb.fit(X_train, y_train)

nbpred = model_nb.predict(X_test)

print('F1 Score',f1_score(y_test, nbpred,average = 'weighted'))
roc_score = multiclass_roc_auc_score(y_test,nbpred)
print("ROC AUC Score - ",roc_score)


# confusion matrix
cm = confusion_matrix(y_test, nbpred)
plt.rcParams['figure.figsize'] = (5, 5)
sns.heatmap(cm, annot = True, cmap = 'twilight')
plt.show()

# classification report
cr = classification_report(y_test, nbpred)
print(cr)
scores = scores.append({'Model': 'GaussianNB', 'Score': roc_score}, ignore_index=True)

In [ ]:
model_u = RandomForestClassifier(n_estimators=100, class_weight='balanced',random_state=0)
model_u.fit(X_train, y_train)

y_pred_rf = model_u.predict(X_test)

print("Training Accuracy: ", model_u.score(X_train, y_train))
print('Testing Accuarcy: ', model_u.score(X_test, y_test))
print('F1 Score',f1_score(y_test, y_pred_rf,average = 'weighted'))
roc_score = multiclass_roc_auc_score(y_test,y_pred_rf)
print("ROC AUC Score - ",roc_score)

# confusion matrix
cm = confusion_matrix(y_test, y_pred_rf)
plt.rcParams['figure.figsize'] = (5, 5)
sns.heatmap(cm, annot = True, cmap = 'winter')
plt.show()

# classification report
cr = classification_report(y_test, y_pred_rf)
print(cr)
scores = scores.append({'Model': 'Random-Forest', 'Score': roc_score}, ignore_index=True)

In [ ]:
model_gb = GradientBoostingClassifier()
model_gb.fit(X_train, y_train)

y_pred_rf = model_gb.predict(X_test)

print("Training Accuracy: ", model_gb.score(X_train, y_train))
print('Testing Accuarcy: ', model_gb.score(X_test, y_test))
print('F1 Score',f1_score(y_test, y_pred_rf,average = 'weighted'))
roc_score = multiclass_roc_auc_score(y_test,y_pred_rf)
print("ROC AUC Score - ",roc_score)

# confusion matrix
cm = confusion_matrix(y_test, y_pred_rf)
plt.rcParams['figure.figsize'] = (5, 5)
sns.heatmap(cm, annot = True, cmap = 'vlag')
plt.show()

# classification report
cr = classification_report(y_test, y_pred_rf)
print(cr)
scores = scores.append({'Model': 'Gradient Boosting', 'Score': roc_score}, ignore_index=True)

In [ ]:
model_xgb = XGBClassifier()
model_xgb.fit(X_train, y_train)

y_pred_rf = model_xgb.predict(X_test)

print("Training Accuracy: ", model_xgb.score(X_train, y_train))
print('Testing Accuarcy: ', model_xgb.score(X_test, y_test))
print('F1 Score',f1_score(y_test, y_pred_rf,average = 'weighted'))
roc_score = multiclass_roc_auc_score(y_test,y_pred_rf)
print("ROC AUC Score - ",roc_score)

# confusion matrix
cm = confusion_matrix(y_test, y_pred_rf)
plt.rcParams['figure.figsize'] = (5, 5)
sns.heatmap(cm, annot = True, cmap = 'vlag_r')
plt.show()

# classification report
cr = classification_report(y_test, y_pred_rf)
print(cr)
scores = scores.append({'Model': 'XGradient Boosting', 'Score': roc_score}, ignore_index=True)

#Snippet for imbalanced Data Using Imblearn Balanced Classifiers.  I don't know if this data is imbalanced or not. I copied the snippet, just in case I need it in the future.

In [ ]:
model_brf = BalancedRandomForestClassifier(n_estimators = 100,max_depth=8, random_state = 0)

model_brf.fit(X_train, y_train)
y_pred_brf = model_brf.predict(X_test)

print("Training Accuracy: ", model_brf.score(X_train, y_train))
print('Testing Accuarcy: ', model_brf.score(X_test, y_test))
print('F1 Score',f1_score(y_test, y_pred_brf,average = 'weighted'))
roc_score = multiclass_roc_auc_score(y_test,y_pred_brf)
print("ROC AUC Score - ",roc_score)


# making a classification report
cr = classification_report(y_test,  y_pred_brf)
print(cr)

# making a confusion matrix
plt.rcParams['figure.figsize'] = (5, 5)
cm = confusion_matrix(y_test, y_pred_brf)
sns.heatmap(cm, annot = True, cmap = 'magma')
plt.show()
scores = scores.append({'Model': 'Balanced RF', 'Score': roc_score}, ignore_index=True)

In [ ]:
model1 = EasyEnsembleClassifier(n_estimators = 100, random_state = 0)

model1.fit(X_train, y_train)
y_pred_ef = model1.predict(X_test)

print("Training Accuracy: ", model1.score(X_train, y_train))
print('Testing Accuarcy: ', model1.score(X_test, y_test))
print('F1 Score',f1_score(y_test, y_pred_ef,average = 'weighted'))
roc_score = multiclass_roc_auc_score(y_test,y_pred_ef)
print("ROC AUC Score - ",roc_score)



# making a classification report
cr = classification_report(y_test,  y_pred_ef)
print(cr)

# making a confusion matrix
cm = confusion_matrix(y_test, y_pred_ef)
sns.heatmap(cm, annot = True, cmap = 'copper')
plt.show()
scores = scores.append({'Model': 'Easy-Ensemble CLF', 'Score': roc_score}, ignore_index=True)

In [ ]:
model2 = BalancedBaggingClassifier(base_estimator = RandomForestClassifier(),
                                 sampling_strategy = 'auto',
                                 replacement = False,
                                 random_state = 0)

model2.fit(X_train, y_train)
y_pred_bc = model2.predict(X_test)

print("Training Accuracy: ", model2.score(X_train, y_train))
print('Testing Accuarcy: ', model2.score(X_test, y_test))
print('F1 Score',f1_score(y_test, y_pred_bc,average = 'weighted'))
roc_score = multiclass_roc_auc_score(y_test,y_pred_bc)
print("ROC AUC Score - ",roc_score)


# making a classification report
cr = classification_report(y_test,  y_pred_bc)
print(cr)

# making a confusion matrix
cm = confusion_matrix(y_test, y_pred_bc)
sns.heatmap(cm, annot = True, cmap = 'Purples')
plt.show()
scores = scores.append({'Model': 'Balanced Bagging Classifier', 'Score': roc_score}, ignore_index=True)

In [ ]:
scores.sort_values(by='Score', ascending=False)

#Hyper Parameter Tuning for Random Forest Model

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)]
max_depth = [int(x) for x in np.linspace(1, 10, num = 10)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

In [ ]:
rf_random = RandomizedSearchCV(estimator=model_u, param_distributions=random_grid,
                              n_iter = 100, scoring='f1_weighted', 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)

rf_random.fit(X_train, y_train)

In [ ]:
rf_random.best_params_

In [ ]:
rf_random.cv_results_

In [ ]:
# fitting up tuned model param.
model_tuned = RandomForestClassifier(n_estimators =  944,min_samples_split = 10,min_samples_leaf= 1,max_depth = None,bootstrap= False)
model_tuned.fit(X_train, y_train)

y_pred_rf = model_tuned.predict(X_test)

print("Training Accuracy: ", model_tuned.score(X_train, y_train))
print('Testing Accuarcy: ', model_tuned.score(X_test, y_test))
print('F1 Score',f1_score(y_test, y_pred_rf,average = 'weighted'))
roc_score = multiclass_roc_auc_score(y_test,y_pred_rf)
print("ROC AUC Score - ",roc_score)

# confusion matrix
cm = confusion_matrix(y_test, y_pred_rf)
plt.rcParams['figure.figsize'] = (5, 5)
sns.heatmap(cm, annot = True, cmap = 'YlOrBr')
plt.show()

# classification report
cr = classification_report(y_test, y_pred_rf)
print(cr)

#Setting up ML Pipeline

Validation of Pipeline created

In [ ]:
model_pipeline = Pipeline(steps=[('scaling',StandardScaler()),
                                 ('RFTuned', RandomForestClassifier(n_estimators =  944,min_samples_split = 10,min_samples_leaf= 1,max_depth = None,bootstrap= False))])
model_pipeline.fit(X_train, y_train)

model_pipeline_pred = model_pipeline.predict(X_test)

print("Training Accuracy: ", model_pipeline.score(X_train, y_train))
print('Testing Accuarcy: ', model_pipeline.score(X_test, y_test))
print('F1 Score',f1_score(y_test, model_pipeline_pred,average = 'weighted'))
roc_score = multiclass_roc_auc_score(y_test,model_pipeline_pred)
print("ROC AUC Score - ",roc_score)

# confusion matrix
cm = confusion_matrix(y_test, model_pipeline_pred)
plt.rcParams['figure.figsize'] = (5, 5)
sns.heatmap(cm, annot = True, cmap = 'RdGy')
plt.show()

# classification report
cr = classification_report(y_test, model_pipeline_pred)
print(cr)

In [ ]:
!pip install dexplot -q

In [ ]:
import dexplot as dxp

dxp.bar(x = 'Model',y = 'Score',data = scores,cmap='geyser',figsize=(10,5),title='Model Score without OHE')

In [ ]:
## Using Random Forest Model For Interpretation
shap_values = shap.TreeExplainer(model_u).shap_values(X_train)

In [ ]:
shap.summary_plot(shap_values, train_cols_list, plot_type="bar")

It's a multi class classification stacked bar plot shows impact of classes on model performance.
I don't know even if this is MultiClass classification or not.